In [14]:
import numpy as np
import pandas as pd
from numba import njit
from my_stuff import MufexKeys
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.indicators.indicators import *
from quantfreedom.plotting.plot_funcs import plot_candles_1_ind_dif_pane, plot_candles_1_ind_same_pane
from quantfreedom.exchanges.exchange import Exchange
np.set_printoptions(formatter={"float_kind": "{:0.4f}".format})
from my_stuff import MufexTestKeys


mufex_test = Mufex(api_key=MufexTestKeys.api_key, secret_key=MufexTestKeys.secret_key, use_test_net=True,)
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
candles = mufex_main.get_candles(symbol="BTCUSDT", timeframe="5m", candles_to_dl=200)
open = candles[:1]
high = candles[:, 2]
low = candles[:, 3]
close = candles[:, 4]
close_shift = np.roll(candles[:, 4],1)

It took 00 mins and 00 seconds to download 200 candles


In [18]:
mufex_test.create_order(
    params={
        "symbol": 'BTCUSDT',
        "positionIdx": 1,
        "side": "Buy",
        "orderType": "Market",
        "qty": str(.01),
        "timeInForce": 'ImmediateOrCancel',
    }
)

'd0b969a5-b2fe-48ce-b1e7-4535830aa25f'